In [1]:
# CELL 1: Check GPU
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# CELL 2: Install Dependencies
print("📦 Installing dependencies...")
!pip install -q ftfy regex tqdm
!pip install -q git+https://github.com/openai/CLIP.git
!pip install -q faiss-gpu-cu12
!pip install -q pillow
!pip install -q matplotlib
!pip install -q requests pandas
print("✅ All dependencies installed!")

📦 Installing dependencies...
  Preparing metadata (setup.py) ... done
✅ All dependencies installed!


In [3]:
# CELL 3: Import Libraries
import torch
import clip
import faiss
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import os
import pickle
import json
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import List, Tuple
import time
import pandas as pd

print("✅ Libraries imported")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

✅ Libraries imported
PyTorch version: 2.9.0+cpu
CUDA available: False


In [4]:
# CELL 4: Load CLIP Model
print("\n🚀 Loading CLIP model...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
print(f"✅ CLIP model loaded on {device}")
print(f"Model: ViT-B/32")
print(f"Input resolution: 224x224")


🚀 Loading CLIP model...
✅ CLIP model loaded on cpu
Model: ViT-B/32
Input resolution: 224x224


In [ ]:
# CELL 5: Download Multiple Small Datasets

print("\n📥 Downloading multiple image datasets...")

import os
from pathlib import Path

# ── 1. COCO val2017 ── (~5,000 images)
print("→ Downloading & extracting COCO val2017...")
os.makedirs("images/coco", exist_ok=True)

# Download if not already present
coco_zip = "val2017.zip"
if not os.path.exists(coco_zip):
    !wget -q http://images.cocodataset.org/zips/val2017.zip -O {coco_zip}

# Extract with overwrite (non-interactive)
!unzip -q -o {coco_zip} -d images/coco > /dev/null 2>&1

# Clean up zip
if os.path.exists(coco_zip):
    os.remove(coco_zip)

coco_count = len(list(Path("images/coco/val2017").glob("*.jpg")))
print(f"   → COCO images extracted: {coco_count}")

# ── 2. Flickr30k images ── (reliable public mirror)
print("→ Downloading Flickr30k images (from awsaf49 GitHub mirror)...")
os.makedirs("images/flickr30k", exist_ok=True)

# Download the three split parts
parts = [
    "flickr30k_part00",
    "flickr30k_part01",
    "flickr30k_part02"
]

for part in parts:
    url = f"https://github.com/awsaf49/flickr-dataset/releases/download/v1.0/{part}"
    !wget -q {url} -O {part}

# Combine into one zip
!cat {' '.join(parts)} > flickr30k.zip 2>/dev/null

# Clean up parts
for part in parts:
    if os.path.exists(part):
        os.remove(part)

# Extract non-interactively
print("   → Extracting Flickr30k...")
!unzip -q -o flickr30k.zip -d flickr_temp > /dev/null 2>&1

# Move images (some mirrors have different folder structure)
!mkdir -p images/flickr30k
!find flickr_temp -type f -name "*.jpg" -exec mv {} images/flickr30k/ \; 2>/dev/null || true

# Clean up temp files
!rm -rf flickr_temp flickr30k.zip

# Limit to first 9000 images (adjust as needed)
flickr_paths = sorted(Path("images/flickr30k").glob("*.jpg"))
flickr_selected = flickr_paths[:9000]
flickr_count = len(flickr_selected)
print(f"   → Using {flickr_count} images from Flickr30k")

# ── Create unified image directory ──
print("→ Creating unified image folder...")
os.makedirs("images/all", exist_ok=True)

# Copy COCO images (non-overwrite)
!cp -n images/coco/val2017/*.jpg images/all/ 2>/dev/null || true

# Copy selected Flickr30k images
for src in flickr_selected:
    !cp -n "{src}" images/all/ 2>/dev/null || true

# Final count
total_images = len(list(Path("images/all").glob("*.jpg")))
print(f"Total images ready for encoding: {total_images}")
print(f"   → Directory: images/all")


📥 Downloading multiple image datasets...
→ Downloading & extracting COCO val2017...
   → COCO images extracted: 5000
→ Downloading Flickr30k images (from awsaf49 GitHub mirror)...
   → Extracting Flickr30k...
   → Using 9000 images from Flickr30k
→ Creating unified image folder...


In [ ]:
import os
from pathlib import Path

# Assuming IMAGE_DIR is still 'images/all'
base_dir = os.path.abspath(IMAGE_DIR)  # e.g. /content/images/all

# Fix all paths to be relative or absolute based on current location
fixed_paths = []
missing_count = 0

print("Fixing image paths...")
for old_path in search_engine.image_paths:
    filename = os.path.basename(old_path)
    new_path = os.path.join(base_dir, filename)

    if os.path.exists(new_path):
        fixed_paths.append(new_path)
    else:
        fixed_paths.append(None)  # or skip, but keep index aligned
        missing_count += 1
        print(f"Still missing: {filename}")

search_engine.image_paths = fixed_paths

print(f"Path fix complete.")
print(f"Total images in index: {len(search_engine.image_paths)}")
print(f"Still missing files: {missing_count}")
print(f"Example fixed path: {search_engine.image_paths[0] if search_engine.image_paths else 'None'}")

In [ ]:
# CELL 6: CLIPImageSearch Class (same as before)

class CLIPImageSearch:
    def __init__(self, model, preprocess, device):
        self.model = model
        self.preprocess = preprocess
        self.device = device
        self.image_paths = []
        self.image_features = None
        self.index = None
        self.dimension = 512

    def encode_images(self, image_dir: str, batch_size: int = 32):
        image_extensions = {'.jpg', '.jpeg', '.png'}
        image_files = []
        for ext in image_extensions:
            image_files.extend(Path(image_dir).glob(f'*{ext}'))

        self.image_paths = [str(p) for p in sorted(image_files)]
        print(f"📁 Found {len(self.image_paths)} images")

        if len(self.image_paths) == 0:
            print("❌ No images found!")
            return

        all_features = []
        for i in tqdm(range(0, len(self.image_paths), batch_size), desc="Encoding images"):
            batch_paths = self.image_paths[i:i+batch_size]
            batch_images = []
            for img_path in batch_paths:
                try:
                    image = Image.open(img_path).convert('RGB')
                    image = self.preprocess(image)
                    batch_images.append(image)
                except:
                    continue

            if not batch_images:
                continue

            batch_tensor = torch.stack(batch_images).to(self.device)
            with torch.no_grad():
                features = self.model.encode_image(batch_tensor)
                features = features / features.norm(dim=-1, keepdim=True)
            all_features.append(features.cpu().numpy())

        if all_features:
            self.image_features = np.vstack(all_features)
            print(f"✅ Encoded {len(self.image_features)} images")
        else:
            print("❌ Encoding failed")

    def build_index(self, index_type='flatip'):
        if self.image_features is None:
            print("❌ No features! Run encode_images first.")
            return

        print(f"Building FAISS index ({index_type})...")

        if index_type == 'flatl2':
            self.index = faiss.IndexFlatL2(self.dimension)
        elif index_type == 'flatip':
            self.index = faiss.IndexFlatIP(self.dimension)
        elif index_type == 'ivf':
            nlist = 100
            quantizer = faiss.IndexFlatL2(self.dimension)
            self.index = faiss.IndexIVFFlat(quantizer, self.dimension, nlist)
            self.index.train(self.image_features)

        self.index.add(self.image_features.astype('float32'))
        print(f"✅ Index built — {self.index.ntotal} vectors")

    def search_by_text(self, query: str, top_k: int = 10) -> List[Tuple[str, float]]:
        text_tokens = clip.tokenize([query]).to(self.device)
        with torch.no_grad():
            text_features = self.model.encode_text(text_tokens)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        text_features = text_features.cpu().numpy().astype('float32')

        distances, indices = self.index.search(text_features, top_k)

        results = []
        for idx, dist in zip(indices[0], distances[0]):
            if idx < len(self.image_paths):
                results.append((self.image_paths[idx], float(dist)))
        return results

    def save_index(self, save_dir: str):
        os.makedirs(save_dir, exist_ok=True)
        faiss.write_index(self.index, os.path.join(save_dir, 'image_index.faiss'))
        metadata = {
            'image_paths': self.image_paths,
            'num_images': len(self.image_paths),
            'dimension': self.dimension
        }
        with open(os.path.join(save_dir, 'metadata.json'), 'w') as f:
            json.dump(metadata, f, indent=2)
        np.save(os.path.join(save_dir, 'image_features.npy'), self.image_features)
        print(f"Index saved → {save_dir}")

print("✅ CLIPImageSearch class ready")

In [ ]:
# CELL 7: Build & Save Index

search_engine = CLIPImageSearch(model, preprocess, device)

print("\nEncoding images (this will take several minutes)...")
search_engine.encode_images(IMAGE_DIR, batch_size=48)   # adjust batch size according to GPU memory

print("\nBuilding index...")
search_engine.build_index(index_type='flatip')          # cosine similarity

print("\nSaving index...")
search_engine.save_index('clip_index_small')

In [ ]:
# CELL 8: Quick Test – Text Search

test_queries = [
    "a cute dog running in the park",
    "person riding a bicycle",
    "beautiful mountain landscape",
    "plate of delicious food",
    "woman wearing red dress"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    results = search_engine.search_by_text(query, top_k=5)
    for i, (path, score) in enumerate(results, 1):
        print(f"  {i}. {os.path.basename(path)}  → score {score:.4f}")

In [ ]:
def show_top_results(query, results, ncols=5):
    print(f"\nQuery: {query}")
    plt.figure(figsize=(15, 4))
    displayed = 0
    for fname, score in results:
        # Use basename only, combine with current IMAGE_DIR
        img_path = os.path.join(IMAGE_DIR, os.path.basename(fname))
        if os.path.exists(img_path):
            img = Image.open(img_path)
            plt.subplot(1, ncols, displayed + 1)
            plt.imshow(img)
            plt.title(f"{score:.4f}")
            plt.axis('off')
            displayed += 1
        else:
            print(f"Missing image: {os.path.basename(fname)}")
    if displayed == 0:
        print("No images found for display")
    plt.suptitle(query, fontsize=14)
    plt.tight_layout()
    plt.show()

# Test again
for q in [
    "a cute dog running in the park",
    "person riding a bicycle",
    "plate of delicious food"
]:
    results = search_engine.search_by_text(q, top_k=5)
    show_top_results(q, results)

In [ ]:
# Save (or re-save) the index
search_engine.save_index("clip_search_index")

# Optional: create a folder structure that's easier to upload
!mkdir -p hf_space_files
!cp -r clip_search_index hf_space_files/
!cp -r images/all hf_space_files/images   # or only a subset if too big

In [ ]:
import json
import os

# Load existing metadata
with open("clip_search_index/metadata.json", "r") as f:
    metadata = json.load(f)

# Convert to filenames only
metadata["image_paths"] = [os.path.basename(p) for p in metadata["image_paths"]]

# Optional: also save num_images
metadata["num_images"] = len(metadata["image_paths"])

# Save updated version
with open("clip_search_index/metadata_clean.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("Saved metadata with filenames only")
print(f"Number of images referenced: {metadata['num_images']}")

In [ ]:
import random
import shutil

subset_dir = "hf_space_files/images_subset"
os.makedirs(subset_dir, exist_ok=True)

all_images = list(Path("images/all").glob("*.jpg"))
selected = random.sample(all_images, min(500, len(all_images)))

for img_path in selected:
    shutil.copy(img_path, subset_dir)

print(f"Copied {len(selected)} images to images_subset/")

In [ ]:
!pip install -U huggingface_hub

from huggingface_hub import login

# Run this cell and follow the link to authenticate
login()   # ← paste your Hugging Face access token here

In [ ]:
from huggingface_hub import HfApi
import os
from pathlib import Path
import random
import shutil

# ==============================================
# CONFIG - change these if needed
# ==============================================
SPACE_ID = "Shaheerkhan/clip-faiss-image-search"   # your space
LOCAL_IMAGE_FOLDER = "images/all"                   # where your full images are
NUMBER_OF_IMAGES_TO_UPLOAD = 300                    # change this (100–800 is reasonable)
PATH_IN_SPACE = "images"                            # folder name in the space

# ==============================================
# Prepare temporary folder for upload
# ==============================================
temp_upload_dir = "temp_images_upload"
os.makedirs(temp_upload_dir, exist_ok=True)

# Get all images
all_images = list(Path(LOCAL_IMAGE_FOLDER).glob("*.jpg"))

if not all_images:
    print("Error: No images found in", LOCAL_IMAGE_FOLDER)
else:
    print(f"Found {len(all_images)} images total")

    # Select random subset
    selected_images = random.sample(all_images, min(NUMBER_OF_IMAGES_TO_UPLOAD, len(all_images)))
    print(f"Selected {len(selected_images)} images to upload")

    # Copy to temp folder
    for img_path in selected_images:
        shutil.copy(img_path, temp_upload_dir)

    print("Temporary folder prepared:", temp_upload_dir)

    # Show first few files (fixed version)
    files = list(Path(temp_upload_dir).glob("*"))
    if files:
        print("First few files:", [f.name for f in files[:5]])
    else:
        print("No files were copied – check if source images exist")

    # ==============================================
    # Upload to Hugging Face Space
    # ==============================================
    api = HfApi()

    print(f"\nUploading {len(selected_images)} images to {SPACE_ID}/{PATH_IN_SPACE}/ ...")

    api.upload_folder(
        folder_path=temp_upload_dir,
        repo_id=SPACE_ID,
        repo_type="space",
        path_in_repo=PATH_IN_SPACE,           # creates images/ folder if needed
        commit_message=f"Add {len(selected_images)} demo images",
        ignore_patterns=["*.tmp", "__pycache__"]
    )

    print("\nUpload finished!")
    print(f"Images should now appear in: https://huggingface.co/spaces/{SPACE_ID}/tree/main/{PATH_IN_SPACE}")

In [ ]:
import json
import os
from pathlib import Path

# Point to your local images folder
image_folder = "images/all"   # or wherever your full set is

# Get all .jpg filenames
image_files = [f.name for f in Path(image_folder).glob("*.jpg")]

# Create metadata structure (filenames only)
metadata = {
    "image_paths": image_files,          # just names like "000000123456.jpg"
    "num_images": len(image_files),
    "dimension": 512                     # CLIP ViT-B/32 dimension
}

# Save it locally
os.makedirs("clip_search_index", exist_ok=True)
with open("clip_search_index/metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print(f"Created metadata.json with {len(image_files)} filenames")
print("First few filenames:", image_files[:5])

In [ ]:
from huggingface_hub import login

login()   # ← just call it like this

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

api.upload_file(
    path_or_fileobj="clip_search_index/metadata.json",
    path_in_repo="clip_search_index/metadata.json",
    repo_id="Shaheerkhan/clip-faiss-image-search",
    repo_type="space",
    commit_message="Add missing metadata.json"
)

In [ ]:
api.upload_folder(
    folder_path="clip_search_index",               # folder with metadata.json + image_index.faiss
    repo_id="Shaheerkhan/clip-faiss-image-search",
    repo_type="space",
    path_in_repo="clip_search_index",
    commit_message="Add FAISS index and metadata"
)

In [ ]:
# Re-run these lines (adjust if your variable name is different)
search_engine.build_index(index_type='flatip')
search_engine.save_index('clip_search_index')

In [ ]:
!ls -lh clip_search_index/

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

# Upload FAISS index
api.upload_file(
    path_or_fileobj="clip_search_index/image_index.faiss",
    path_in_repo="clip_search_index/image_index.faiss",
    repo_id="Shaheerkhan/clip-faiss-image-search",
    repo_type="space",
    commit_message="Add missing image_index.faiss"
)

# Re-upload metadata.json (to make sure it's fresh)
api.upload_file(
    path_or_fileobj="clip_search_index/metadata.json",
    path_in_repo="clip_search_index/metadata.json",
    repo_id="Shaheerkhan/clip-faiss-image-search",
    repo_type="space",
    commit_message="Update metadata.json"
)

print("Upload complete! Wait 2–5 minutes then refresh Space.")

In [ ]:
!ls -lh clip_search_index/
!du -sh clip_search_index/image_index.faiss 2>/dev/null || echo "image_index.faiss not found!"

In [ ]:
import torch
import clip
from pathlib import Path
from PIL import Image
import numpy as np
from tqdm import tqdm
import faiss
import os
import json

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model, preprocess = clip.load("ViT-B/32", device=device)

class CLIPImageSearch:
    def __init__(self, model, preprocess, device):
        self.model = model
        self.preprocess = preprocess
        self.device = device
        self.image_paths = []
        self.image_features = None
        self.index = None
        self.dimension = 512

    def encode_images(self, image_dir, batch_size=16):
        files = list(Path(image_dir).glob("*.jpg"))
        if not files:
            print("No images found in", image_dir)
            return

        self.image_paths = [str(p) for p in files]
        print(f"Found {len(self.image_paths)} images")

        features_list = []
        for start in tqdm(range(0, len(files), batch_size)):
            batch_paths = files[start:start+batch_size]
            batch_imgs = []
            for p in batch_paths:
                try:
                    img = Image.open(p).convert("RGB")
                    batch_imgs.append(self.preprocess(img))
                except Exception as e:
                    print(f"Skip {p}: {e}")
            if not batch_imgs:
                continue
            tensor = torch.stack(batch_imgs).to(self.device)
            with torch.no_grad():
                emb = self.model.encode_image(tensor)
                emb = emb / emb.norm(dim=-1, keepdim=True)
            features_list.append(emb.cpu().numpy())

        if features_list:
            self.image_features = np.concatenate(features_list)
            print(f"Encoded {len(self.image_features)} features successfully")
        else:
            print("No features encoded - check image loading")

    def build_index(self):
        if self.image_features is None or len(self.image_features) == 0:
            print("No features to index!")
            return
        self.index = faiss.IndexFlatIP(self.dimension)
        self.index.add(self.image_features.astype('float32'))
        print(f"Index built with {self.index.ntotal} vectors")

    def save_index(self, save_dir):
        os.makedirs(save_dir, exist_ok=True)
        if self.index is not None:
            faiss.write_index(self.index, os.path.join(save_dir, "image_index.faiss"))
        metadata = {
            "image_paths": self.image_paths,
            "num_images": len(self.image_paths),
            "dimension": self.dimension
        }
        with open(os.path.join(save_dir, "metadata.json"), "w") as f:
            json.dump(metadata, f, indent=2)
        print(f"Saved to {save_dir}")
        !ls -lh {save_dir}

print("Class ready")

In [ ]:
search_engine = CLIPImageSearch(model, preprocess, device)

# Use the subset you uploaded or full folder
# Change path if needed
search_engine.encode_images("temp_images_upload", batch_size=16)  # or "images/all"

In [ ]:
from pathlib import Path

possible_dirs = [
    "images/all",
    "images",
    "temp_images_upload",
    "hf_space_files/images_subset",
    "Images",
    "images/coco/val2017",
    "images/flickr30k",
    "images/unsplash"
]

for d in possible_dirs:
    if Path(d).exists():
        jpg_count = len(list(Path(d).glob("*.jpg")))
        print(f"Folder '{d}': {jpg_count} .jpg files")
    else:
        print(f"Folder '{d}': does NOT exist")

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

api.upload_file(
    path_or_fileobj="clip_search_index/image_index.faiss",
    path_in_repo="clip_search_index/image_index.faiss",
    repo_id="Shaheerkhan/clip-faiss-image-search",
    repo_type="space",
    commit_message="Upload missing image_index.faiss"
)